In [ ]:
'''
A Convolutional Neural Network implementaion for handwritten digits classification.
This example is using Tensorflow library.
The dataset used is MNIST database of handwritten digits.
(http://yann.lecun.com/exdb/mnist/)
'''

In [1]:
import tensorflow as tf

#Import MNIST dataset that has all the images
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
tf.__version__

'1.0.0'

In [3]:
#Set the parameters..
learning_rate = 0.001
training_iterations = 200000
batch_size = 128
display_step = 10
model_path = '/tmp/model.ckpt'

#Network model parameters..
n_input = 784 # MNIST data input (image shape = 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 #probability to keep the output units

#tensorflow graph input..
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout

In [4]:
# make functions for the operations..

def conv_2d(x, w, b, strides=1):
    x = tf.nn.conv2d(x, w, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool_2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

#build the model..

def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    #convolutional layer
    conv_1 = conv_2d(x, weights['wc1'], biases['bc1'])
    #max pooling
    conv_1 = maxpool_2d(conv_1, k=2)
    
    #2nd convolutional layer
    conv_2 = conv_2d(conv_1, weights['wc2'], biases['bc2'])
    #2nd maxpool layer
    conv_2 = maxpool_2d(conv_2, k=2)
    
    #fully connected layers
    #reshaping the convolutional layer output
    fc_1 = tf.reshape(conv_2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc_1 = tf.add(tf.matmul(fc_1, weights['wd1']), biases['bd1'])
    fc_1 = tf.nn.relu(fc_1)
    
    #apply dropout
    fc_1 = tf.nn.dropout(fc_1, dropout)
    
    #output, class prediction
    out = tf.add(tf.matmul(fc_1, weights['out']), biases['out'])
    
    return out
    

In [5]:
#make dictionaries of the weights and biases
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7x7x64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # fully vonnected 1024 inputs, 10 outputs
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

#construct the model..
pred = conv_net(x, weights, biases, keep_prob)

#loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialize the variables
init = tf.global_variables_initializer()


In [6]:
#Save op to save and restore all the variables
saver = tf.train.Saver()

In [7]:
from time import time

In [12]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    start = time()
    
    while step * batch_size < training_iterations:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization op
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.75})
        
        if step % display_step == 0:
            #calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            
            print('Iter '+str(step*batch_size)+', Minibatch Loss = '+ '{:.5f}'.format(loss)+', Training Accuracy = '+ '{:.5f}'.format(acc))
            
        step += 1
            
    stop = time()
    print('\nOptimization Finished!')
    print('\nTime taken to train: ', stop-start)
    
    #calculate accuracy for 512 mnist test images
    print('\nTesting Accuracy: ', sess.run(accuracy, feed_dict={x: mnist.test.images[:512], y: mnist.test.labels[:512], keep_prob: 1.}))
    
    #save model weights to disk
    #save_path = saver.save(sess, model_path)
    #print('Model saved in file: %s' %save_path)

Iter 1280, Minibatch Loss = 31679.54102, Training Accuracy = 0.26562
Iter 2560, Minibatch Loss = 15754.62500, Training Accuracy = 0.45312
Iter 3840, Minibatch Loss = 9969.21484, Training Accuracy = 0.61719
Iter 5120, Minibatch Loss = 6388.72314, Training Accuracy = 0.67188
Iter 6400, Minibatch Loss = 8468.33691, Training Accuracy = 0.66406
Iter 7680, Minibatch Loss = 5249.65283, Training Accuracy = 0.76562
Iter 8960, Minibatch Loss = 3926.67676, Training Accuracy = 0.85156
Iter 10240, Minibatch Loss = 1991.85706, Training Accuracy = 0.88281
Iter 11520, Minibatch Loss = 1965.12122, Training Accuracy = 0.87500
Iter 12800, Minibatch Loss = 5759.95605, Training Accuracy = 0.82031
Iter 14080, Minibatch Loss = 2854.27148, Training Accuracy = 0.86719
Iter 15360, Minibatch Loss = 2022.49011, Training Accuracy = 0.87500
Iter 16640, Minibatch Loss = 2277.18726, Training Accuracy = 0.85156
Iter 17920, Minibatch Loss = 2164.70752, Training Accuracy = 0.88281
Iter 19200, Minibatch Loss = 1784.49512